![](https://i.imgur.com/vDsZKLt.jpeg)

In [1]:
import pandas as pd
import sqlite3

In [2]:
credits_df = pd.read_csv('/kaggle/input/netflix-tv-shows-and-movies/credits.csv')
titles_df = pd.read_csv('/kaggle/input/netflix-tv-shows-and-movies/titles.csv')
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
credits_df.to_sql('credits', conn, if_exists='replace', index=False)
titles_df.to_sql('titles', conn, if_exists='replace', index=False)
print("✅ Successfully loaded the datasets into the SQLite database.")

✅ Successfully loaded the datasets into the SQLite database.


In [3]:
cursor.execute("""
CREATE TABLE titles_clean AS
SELECT *
FROM titles
WHERE 
    title IS NOT NULL
    AND type IN ('MOVIE', 'SHOW')
    AND imdb_score IS NOT NULL
    AND tmdb_score IS NOT NULL
    AND runtime > 0
""")
print("✅ Created 'titles_clean' table.")
cursor.execute("""
CREATE TABLE credits_clean AS
SELECT *
FROM credits
WHERE 
    id IS NOT NULL
    AND name IS NOT NULL
    AND role IN ('ACTOR', 'DIRECTOR')
""")
print("✅ Created 'credits_clean' table.")
cursor.execute("CREATE INDEX idx_titles_clean_id ON titles_clean(id)")
cursor.execute("CREATE INDEX idx_credits_clean_id ON credits_clean(id)")
print("✅ Created indexes on 'titles_clean' and 'credits_clean'.")
print("🎉 Preprocessing complete. Ready for querying!")

✅ Created 'titles_clean' table.
✅ Created 'credits_clean' table.
✅ Created indexes on 'titles_clean' and 'credits_clean'.
🎉 Preprocessing complete. Ready for querying!


In [4]:
query = """
SELECT title, type, imdb_score 
FROM titles_clean
WHERE type = 'MOVIE'
ORDER BY imdb_score DESC
LIMIT 10
"""
Top_10_Movies_by_IMDB_Score = pd.read_sql(query, conn)
Top_10_Movies_by_IMDB_Score

,title,type,imdb_score
0,Major,MOVIE,9.1
1,C/o Kancharapalem,MOVIE,8.9
2,David Attenborough: A Life on Our Planet,MOVIE,8.9
3,Forrest Gump,MOVIE,8.8
4,Inception,MOVIE,8.8
5,GoodFellas,MOVIE,8.7
6,Anbe Sivam,MOVIE,8.7
7,A Lion in the House,MOVIE,8.7
8,Chhota Bheem & Krishna in Mayanagari,MOVIE,8.7
9,Bo Burnham: Inside,MOVIE,8.7


In [5]:
query = """
SELECT title, type, imdb_score
FROM titles_clean
WHERE type = 'MOVIE' 
    AND imdb_score IS NOT NULL
ORDER BY imdb_score ASC
LIMIT 10
"""
Bottom_10_Movies_by_IMDB_Score = pd.read_sql(query, conn)
Bottom_10_Movies_by_IMDB_Score

,title,type,imdb_score
0,Me Against You: Mr. S's Vendetta,MOVIE,1.6
1,Himmatwala,MOVIE,1.7
2,Kyaa Kool Hain Hum 3,MOVIE,1.9
3,FRED 3: Camp Fred,MOVIE,2.0
4,Hajwala: The Missing Engine,MOVIE,2.0
5,Dostana,MOVIE,2.1
6,B. A. Pass 2,MOVIE,2.2
7,Time to Dance,MOVIE,2.2
8,Nothing to Lose,MOVIE,2.3
9,Bobbleheads: The Movie,MOVIE,2.3


In [6]:
query = """
SELECT 'Longest Runtime' AS type, 
       title, 
       runtime
FROM titles_clean
WHERE runtime = (SELECT MAX(runtime) 
                 FROM titles_clean
                 WHERE type = 'MOVIE')
UNION
SELECT 'Shortest Runtime' AS type, 
       title, 
       runtime
FROM titles_clean
WHERE runtime = (SELECT MIN(runtime) 
                 FROM titles_clean
                 WHERE runtime <> 0
                     AND type = 'MOVIE')
"""
Longest_and_Shortest_Runtime_Movies = pd.read_sql(query, conn)
Longest_and_Shortest_Runtime_Movies

,type,title,runtime
0,Longest Runtime,A Lion in the House,225
1,Shortest Runtime,Angry Birds Toons,2
2,Shortest Runtime,Dance & Sing With True,2
3,Shortest Runtime,Julie and the Phantoms,2
4,Shortest Runtime,Larva,2
5,Shortest Runtime,Pui Pui Molcar,2
6,Shortest Runtime,Time to Dance,2


In [7]:
query = """
SELECT title, imdb_score, tmdb_score, 
       ROUND((imdb_score + tmdb_score) / 2.0, 2) AS avg_score
FROM titles_clean
WHERE type = 'MOVIE'
  AND imdb_score IS NOT NULL
  AND tmdb_score IS NOT NULL
ORDER BY avg_score DESC
LIMIT 5
"""
Top_5_Movies_with_Highest_Combined_IMDB_and_TMDB_Scores = pd.read_sql(query, conn)
Top_5_Movies_with_Highest_Combined_IMDB_and_TMDB_Scores

,title,imdb_score,tmdb_score,avg_score
0,Breaded Life,7.7,10.0,8.85
1,"Best Wishes, Warmest Regards: A Schitt's Creek...",8.6,8.9,8.75
2,I'm Glad I Did,8.4,9.1,8.75
3,David Attenborough: A Life on Our Planet,8.9,8.5,8.70
4,Emicida AmarElo Live in Sao Paulo,7.6,9.8,8.70


In [8]:
query = """
SELECT release_year, 
       COUNT(*) AS TOTAL
FROM titles_clean
WHERE type = 'MOVIE'
GROUP BY release_year
ORDER BY release_year DESC
LIMIT 5
"""
Movies_Released_by_Year = pd.read_sql(query, conn)
Movies_Released_by_Year

,release_year,TOTAL
0,2022,177
1,2021,401
2,2020,398
3,2019,442
4,2018,431


In [9]:
query = """
SELECT title, type, imdb_score
FROM titles_clean
WHERE type = 'SHOW'
ORDER BY imdb_score DESC
LIMIT 10
"""
Top_10_Shows_by_IMDB_Score = pd.read_sql(query, conn)
Top_10_Shows_by_IMDB_Score

,title,type,imdb_score
0,Breaking Bad,SHOW,9.5
1,Avatar: The Last Airbender,SHOW,9.3
2,Our Planet,SHOW,9.3
3,Reply 1988,SHOW,9.2
4,My Mister,SHOW,9.1
5,The Last Dance,SHOW,9.1
6,Kota Factory,SHOW,9.1
7,Okupas,SHOW,9.0
8,DEATH NOTE,SHOW,9.0
9,Hunter x Hunter,SHOW,9.0


In [10]:
query = """
SELECT title, type, imdb_score
FROM titles_clean
WHERE type = 'SHOW' 
    AND imdb_score IS NOT NULL
ORDER BY imdb_score ASC
LIMIT 10
"""
Bottom_10_Shows_by_IMDB_Score = pd.read_sql(query,conn)
Bottom_10_Shows_by_IMDB_Score

,title,type,imdb_score
0,Thomas & Friends: All Engines Go!,SHOW,2.0
1,He's Expecting,SHOW,2.0
2,Hype House,SHOW,2.1
3,A House of Blocks,SHOW,2.3
4,Until Dawn,SHOW,2.4
5,The Goop Lab,SHOW,2.5
6,Byron Baes,SHOW,2.6
7,Bonus Family,SHOW,2.9
8,Richie Rich,SHOW,3.0
9,Cloudy with a Chance of Meatballs,SHOW,3.4


In [11]:
query = """
SELECT 'Longest Runtime' AS type, 
       title, 
       runtime
FROM titles_clean
WHERE runtime = (SELECT MAX(runtime) 
                 FROM titles_clean
                 WHERE type = 'SHOW')
UNION
SELECT 'Shortest Runtime' AS type, 
       title, 
       runtime
FROM titles_clean
WHERE runtime = (SELECT MIN(runtime) 
                 FROM titles_clean
                 WHERE runtime <> 0
                     AND type = 'SHOW')
"""
Longest_and_Shortest_Runtime_Shows = pd.read_sql(query, conn)
Longest_and_Shortest_Runtime_Shows

,type,title,runtime
0,Longest Runtime,1994,178
1,Shortest Runtime,Angry Birds Toons,2
2,Shortest Runtime,Dance & Sing With True,2
3,Shortest Runtime,Julie and the Phantoms,2
4,Shortest Runtime,Larva,2
5,Shortest Runtime,Pui Pui Molcar,2
6,Shortest Runtime,Time to Dance,2


In [12]:
query = """
SELECT title, 
       seasons
FROM titles_clean
WHERE type = 'SHOW'
ORDER BY seasons DESC
LIMIT 5
"""
Shows_with_Most_Seasons = pd.read_sql(query, conn)
Shows_with_Most_Seasons

,title,seasons
0,Survivor,42.0
1,Wheel of Fortune,39.0
2,The Challenge,37.0
3,Top Gear,32.0
4,Power Rangers,29.0


In [13]:
query = """
SELECT release_year, 
       COUNT(*) AS TOTAL
FROM titles_clean
WHERE type = 'SHOW'
GROUP BY release_year
ORDER BY release_year DESC
"""
Shows_Released_by_Year = pd.read_sql(query, conn)
Shows_Released_by_Year

,release_year,TOTAL
0,2022,151
1,2021,273
2,2020,249
3,2019,275
4,2018,279
5,2017,167
6,2016,124
7,2015,81
8,2014,40
9,2013,33


In [14]:
query = """
SELECT type, 
       ROUND(AVG(imdb_score), 2) AS AVG_imdb_score,
       ROUND(AVG(tmdb_score), 2) AS AVG_tmdb_score
FROM titles_clean
GROUP BY type
"""
Average_IMDB_and_TMDB_Scores_for_Movies_and_Shows = pd.read_sql(query, conn)
Average_IMDB_and_TMDB_Scores_for_Movies_and_Shows

,type,AVG_imdb_score,AVG_tmdb_score
0,MOVIE,6.25,6.46
1,SHOW,6.98,7.47


In [15]:
query = """
SELECT production_countries,
       ROUND(AVG(imdb_score), 2) AS AVG_imdb_score,
       ROUND(AVG(tmdb_score), 2) AS AVG_tmdb_score
FROM titles_clean
GROUP BY production_countries
ORDER BY AVG_imdb_score DESC
"""
Average_IMDB_and_TMDB_Scores_by_Production_Country = pd.read_sql(query, conn)
Average_IMDB_and_TMDB_Scores_by_Production_Country

,production_countries,AVG_imdb_score,AVG_tmdb_score
0,"['UA', 'GB', 'US']",8.3,8.0
1,"['CU', 'US']",8.2,7.7
2,"['CD', 'GB']",8.2,8.0
3,"['ZA', 'GB']",8.1,8.0
4,"['US', 'EG', 'GB']",8.1,7.8
...,...,...,...
414,"['JP', 'US', 'CA']",3.8,5.7
415,"['MX', 'CO']",3.7,5.3
416,"['CN', 'DE', 'GB']",3.7,5.9
417,"['US', 'CY']",2.9,5.0


In [16]:
query = """
SELECT age_certification,
       ROUND(AVG(imdb_score), 2) AS AVG_imdb_score,
       ROUND(AVG(tmdb_score), 2) AS AVG_tmdb_score
FROM titles_clean
GROUP BY age_certification
ORDER BY AVG_imdb_score DESC
"""
Average_IMDB_and_TMDB_Scores_by_Age_Certification = pd.read_sql(query, conn)
Average_IMDB_and_TMDB_Scores_by_Age_Certification

,age_certification,AVG_imdb_score,AVG_tmdb_score
0,TV-14,7.23,7.56
1,TV-MA,7.02,7.34
2,TV-PG,6.92,7.42
3,TV-Y7,6.84,7.77
4,TV-Y,6.54,7.83
5,PG-13,6.44,6.59
6,TV-G,6.32,7.52
7,R,6.30,6.49
8,None,6.26,6.50
9,G,6.25,6.45


In [17]:
query = """
SELECT imdb_score, 
       COUNT(*)
FROM titles_clean
WHERE imdb_score BETWEEN 7 AND 8
GROUP BY imdb_score
ORDER BY imdb_score DESC
"""
Movies_with_IMDB_Scores_between_7_8 = pd.read_sql(query, conn)
Movies_with_IMDB_Scores_between_7_8

,imdb_score,COUNT(*)
0,8.0,89
1,7.9,90
2,7.8,108
3,7.7,118
4,7.6,154
5,7.5,140
6,7.4,191
7,7.3,175
8,7.2,182
9,7.1,189


In [18]:
query = """
SELECT imdb_score, 
       AVG(runtime) AS AVG_Runtime
FROM titles_clean
GROUP BY imdb_score
ORDER BY imdb_score DESC
"""
Average_Runtime_by_IMDB_Score_Range = pd.read_sql(query, conn)
Average_Runtime_by_IMDB_Score_Range

,imdb_score,AVG_Runtime
0,9.5,48.0
1,9.3,37.0
2,9.2,92.0
3,9.1,77.0
4,9.0,33.0
...,...,...
73,2.1,96.5
74,2.0,53.5
75,1.9,124.0
76,1.7,150.0


In [19]:
query = """
SELECT name, 
       COUNT(*) AS Apperance
FROM credits_clean
WHERE role = 'ACTOR'
GROUP BY name
ORDER BY Apperance DESC
LIMIT 20
"""
Top_20_Actors_by_Appearances = pd.read_sql(query, conn)
Top_20_Actors_by_Appearances

,name,Apperance
0,Kareena Kapoor Khan,25
1,Boman Irani,25
2,Shah Rukh Khan,23
3,Takahiro Sakurai,21
4,Priyanka Chopra Jonas,20
5,Paresh Rawal,20
6,Amitabh Bachchan,20
7,Yuki Kaji,19
8,Nawazuddin Siddiqui,19
9,Junichi Suwabe,19


In [20]:
query = """
SELECT name, 
       COUNT(*) AS Apperance
FROM credits_clean
WHERE role = 'DIRECTOR'
GROUP BY name
ORDER BY Apperance DESC
LIMIT 20
"""
Top_20_Directors_by_Appearances = pd.read_sql(query, conn)
Top_20_Directors_by_Appearances

,name,Apperance
0,Raúl Campos,20
1,Jan Suter,19
2,Ryan Polito,17
3,Jay Karas,15
4,Marcus Raboy,14
5,Jay Chapman,12
6,Cathy Garcia-Molina,12
7,Youssef Chahine,11
8,Troy Miller,8
9,Suhas Kadav,8


In [21]:
query = """
SELECT genres, 
       COUNT(*) AS COUNT
FROM titles_clean
GROUP BY genres
ORDER BY COUNT DESC
LIMIT 20
"""
Count_of_Titles_by_Genre_Top_20 = pd.read_sql(query, conn)
Count_of_Titles_by_Genre_Top_20

,genres,COUNT
0,['comedy'],423
1,['documentation'],245
2,['drama'],231
3,"['drama', 'romance']",118
4,"['comedy', 'drama']",114
5,"['comedy', 'documentation']",104
6,"['drama', 'comedy']",96
7,['reality'],95
8,"['comedy', 'drama', 'romance']",65
9,"['comedy', 'romance']",64


In [22]:
query = """
SELECT age_certification, 
       COUNT(*) AS 'COUNT'
FROM titles_clean
WHERE type = 'MOVIE'
GROUP BY age_certification
ORDER BY COUNT DESC
LIMIT 5
"""
Most_Common_Age_Certifications_for_Movies = pd.read_sql(query, conn)
Most_Common_Age_Certifications_for_Movies

,age_certification,COUNT
0,None,2002
1,R,528
2,PG-13,431
3,PG,221
4,G,88


In [23]:
query = """
SELECT t.title, 
       t.release_year, 
       c.role
FROM credits_clean AS c
JOIN titles AS t
    ON c.id = t.id
WHERE c.role = 'DIRECTOR'
    AND t.type = 'MOVIE'
    AND t.release_year >= 2010
ORDER BY t.release_year
"""
Titles_and_Directors_of_Movies_Released_After_2010 = pd.read_sql(query, conn)
Titles_and_Directors_of_Movies_Released_After_2010

,title,release_year,role
0,Inception,2010,DIRECTOR
1,Insidious,2010,DIRECTOR
2,Grown Ups,2010,DIRECTOR
3,The Fighter,2010,DIRECTOR
4,The Book of Eli,2010,DIRECTOR
...,...,...,...
3419,"Rodrigo Sant'anna: I'm Here, I'm Queer!",2022,DIRECTOR
3420,Celeb Five: Behind the Curtain,2022,DIRECTOR
3421,Celeb Five: Behind the Curtain,2022,DIRECTOR
3422,Kongsi Raya,2022,DIRECTOR


In [24]:
query = """
SELECT release_year, 
       COUNT(*) AS TOTAL
FROM titles_clean
GROUP BY release_year
ORDER BY TOTAL DESC
"""
Total_Number_of_Titles_by_Year = pd.read_sql(query, conn)
Total_Number_of_Titles_by_Year

,release_year,TOTAL
0,2019,717
1,2018,710
2,2021,674
3,2020,647
4,2017,523
...,...,...
57,1961,1
58,1960,1
59,1959,1
60,1958,1


In [25]:
query = """
SELECT c.name, 
       c.character, 
       COUNT(DISTINCT t.title) AS how_many_times
FROM titles_clean AS t
JOIN credits AS c
    ON t.id = c.id
WHERE c.character IS NOT NULL
GROUP BY c.name, 
         c.character
HAVING COUNT(DISTINCT t.title) > 1
ORDER BY how_many_times DESC
"""
Actors_Who_Played_Same_Character_in_Multiple_Shows = pd.read_sql(query, conn)
Actors_Who_Played_Same_Character_in_Multiple_Shows

,name,character,how_many_times
0,Chie Nakamura,Sakura Haruno (voice),9
1,Donald Trump,Self (archive footage),9
2,John Paul Tremblay,Julian,9
3,Mike Smith,Bubbles,9
4,Junko Takeuchi,Naruto Uzumaki (voice),8
...,...,...,...
1098,Zoe Saldaña,Herself,2
1099,Álvaro Cervantes,Dr. Berasategui,2
1100,İbrahim Büyükak,Şeref,2
1101,İnan Ulaş Torun,Tarık,2


In [26]:
query = """
SELECT type, 
       COUNT(*) AS count
FROM titles_clean
GROUP BY type
"""
Count_and_List_of_Movies_Shows = pd.read_sql(query, conn)
Count_and_List_of_Movies_Shows

,type,count
0,MOVIE,3285
1,SHOW,1852
